In [1]:
import numpy as np
import pandas as pd
import os
import datetime

import configparser
from jqdatasdk import *

from stock_chart import kline_chart

class StockData():
    
    def __init__(self):
        self.load_user_config()
    
    def load_user_config(self):
        '''Add user information for getting stocks' data
        '''

        cf = configparser.ConfigParser()
        cf.read('../finance/config.ini')
        account = cf.get('user', 'account')
        password = cf.get('user', 'password')
        auth(account,password)
        
    
    def get_price(self, code, start_date='2005-01-01', end_date='2018-11-30'):
        '''Download stock data (adjusted close)
        
        Parameters
        ----------------------
        code: str or int
            code of stock
        start_date: str
            start_date
        end_date: str
            end_date
            
        Returns
        ----------------------
        normalized_code, pd.DataFrame
        '''
        try:
            normalized_code = self.normalize_code(str(code))
        except Exception as e:
            print(e)
            return [], None
        
        if normalized_code == "000001.XSHE":
            price_df = pd.read_csv("data/000001.SXHE.csv",index_col=0)
            price_df = price_df.loc[start_date:end_date,:]
        else:
            security_info = get_security_info(normalized_code)
            if security_info is None:
                return [], None
            security_start_date = security_info.start_date
            security_end_date = security_info.end_date

            # compare security date and requery date
            if datetime.datetime.strptime(start_date,"%Y-%m-%d").date() < security_start_date:
                start_date = security_start_date
            if datetime.datetime.strptime(end_date,"%Y-%m-%d").date() > security_end_date:
                end_date = security_end_date

            price_df = get_price(normalized_code, start_date=start_date, end_date=end_date, frequency='daily', fields=None, 
                                      skip_paused=False, fq='pre')
        self.fill_missing_values(price_df)
        
        return normalized_code, price_df

    def query_prices(self,query_str,start_date=None,end_date=None):

        # split query str
        query_codes = query_str.split(",")

        if len(query_codes) == 1:

            normalized_code, price_df = self.get_price(query_codes[0],start_date,end_date)
            if normalized_code:
                return [normalized_code], price_df, []
            else:
                print('Could not find price data of this code: %s' %query_codes[0])
                return [], None, [query_codes[0]]

        elif len(query_codes) >= 2:

            normalized_codes = []
            price_dfs = {}
            err_codes = []
            
            for code in query_codes:

                normalized_code, price_df = self.get_price(code,start_date,end_date)

                if normalized_code:
                    price_dfs[normalized_code] = price_df
                    normalized_codes.append(normalized_code)
                else:
                    print('Could not find price data of this code: %s' %code)
                    err_codes.append(code)
            
            if len(normalized_codes) == 0:
                return [], None, err_codes
            else:
                return normalized_codes, price_dfs,err_codes
        else:
            pass          


    def normalize_code(self,code):
        '''Normalize code'''
        return normalize_code(code)
        

    def fill_missing_values(self,df_data):
        """Fill missing values in data frame, in place."""
        df_data.fillna(method="ffill", inplace=True)
        df_data.fillna(method="bfill", inplace=True)

In [2]:
sd = StockData()
price_000001 = sd.get_price('000001')
price_000001

auth success


('000001.XSHE',
              open  close   high    low       volume         money
 2005-01-04   1.53   1.51   1.53   1.50    7594032.0  1.146560e+07
 2005-01-05   1.51   1.50   1.52   1.47   13896309.0  2.071856e+07
 2005-01-06   1.51   1.51   1.53   1.50   11499579.0  1.733384e+07
 2005-01-07   1.53   1.51   1.53   1.50    8134502.0  1.230285e+07
 2005-01-10   1.51   1.53   1.53   1.48   11351401.0  1.711150e+07
 2005-01-11   1.53   1.53   1.53   1.53          0.0  0.000000e+00
 2005-01-12   1.53   1.51   1.53   1.50    7067174.0  1.067264e+07
 2005-01-13   1.51   1.52   1.53   1.51    6903742.0  1.050459e+07
 2005-01-14   1.53   1.51   1.53   1.50    6920976.0  1.047666e+07
 2005-01-17   1.50   1.45   1.50   1.44   13677493.0  2.003997e+07
 2005-01-18   1.45   1.44   1.46   1.43   11177731.0  1.610962e+07
 2005-01-19   1.44   1.43   1.45   1.43    7268687.0  1.041656e+07
 2005-01-20   1.43   1.39   1.43   1.39   17030792.0  2.381501e+07
 2005-01-21   1.39   1.49   1.51   1.36   4329

In [58]:
price_szzs = sd.get_price("000300.XSHG")

In [59]:
price_szzs

,open,close,high,low,volume,money
2005-04-08,984.665,1003.445,1003.699,979.529,1.476253e+09,9.151350e+09
2005-04-11,1003.879,995.421,1008.735,992.773,1.593607e+09,1.043623e+10
2005-04-12,993.711,978.697,993.711,978.204,1.022619e+09,6.479563e+09
2005-04-13,987.950,1000.900,1006.499,987.950,1.607169e+09,1.002960e+10
2005-04-14,1004.640,986.975,1006.416,985.578,1.294571e+09,7.813425e+09
2005-04-15,982.613,974.082,982.613,971.930,1.040896e+09,6.541675e+09
2005-04-18,970.907,963.772,970.907,958.649,8.598400e+08,5.329139e+09
2005-04-19,962.916,965.886,968.867,957.910,9.212623e+08,5.768908e+09
2005-04-20,964.151,950.867,964.151,946.197,8.850705e+08,5.300004e+09
2005-04-21,948.861,943.981,955.553,938.595,9.946145e+08,6.140646e+09


In [6]:
all_securities = get_all_securities(types=[], date=None)

In [57]:
all_securities

,display_name,name,start_date,end_date,type
000001.XSHE,平安银行,PAYH,1991-04-03,2200-01-01,stock
000002.XSHE,万科A,WKA,1991-01-29,2200-01-01,stock
000004.XSHE,国农科技,GNKJ,1990-12-01,2200-01-01,stock
000005.XSHE,世纪星源,SJXY,1990-12-10,2200-01-01,stock
000006.XSHE,深振业A,SZYA,1992-04-27,2200-01-01,stock
000007.XSHE,全新好,QXH,1992-04-13,2200-01-01,stock
000008.XSHE,神州高铁,SZGT,1992-05-07,2200-01-01,stock
000009.XSHE,中国宝安,ZGBA,1991-06-25,2200-01-01,stock
000010.XSHE,美丽生态,SHX,1995-10-27,2200-01-01,stock
000011.XSHE,深物业A,SWYA,1992-03-30,2200-01-01,stock


In [18]:
security = '603999.XSHG'
daily_price = {}
df = get_price(security, start_date='1991-12-10', end_date='2018-11-30', frequency='daily', fields=None, 
                                  skip_paused=False, fq='pre')
df

,open,close,high,low,volume,money
2005-01-04,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-10,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-11,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-12,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-13,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-14,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-17,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
get_security_info(security)

603999.XSHG

In [31]:
datetime.strptime("2018-01-01","%Y-%m-%d").date()

datetime.date(2018, 1, 1)

In [63]:
kline_chart(price_szzs,"深证300指数").render_embed()

Markup('\n        <div id="0c9c7c20702a4e83830d2132631dfb0c" style="width:800px;height:400px;"></div>\n        <script type="text/javascript">\n\nvar myChart_0c9c7c20702a4e83830d2132631dfb0c = echarts.init(document.getElementById(\'0c9c7c20702a4e83830d2132631dfb0c\'), \'light\', {renderer: \'canvas\'});\n\nvar option_0c9c7c20702a4e83830d2132631dfb0c = {\n    "title": [\n        {\n            "text": "K \\u7ebf\\u56fe\\u793a\\u4f8b",\n            "left": "auto",\n            "top": "auto",\n            "textStyle": {\n                "fontSize": 18\n            },\n            "subtextStyle": {\n                "fontSize": 12\n            }\n        }\n    ],\n    "toolbox": {\n        "show": true,\n        "orient": "vertical",\n        "left": "95%",\n        "top": "center",\n        "feature": {\n            "saveAsImage": {\n                "show": true,\n                "title": "\\u4e0b\\u8f7d\\u56fe\\u7247"\n            },\n            "restore": {\n                "show": tru

In [69]:
bbb = sd.get_price('000002',start_date='2015-12-18',end_date='2016-07-06')

In [12]:
query='000003'
normalized_name ,price_query = sd.get_price(query)
chart_query = kline_chart(price_query, sd.normalize_code(query))

In [15]:
normalized_name

[]

In [86]:
normalize_code(str('000015'))

'000015.XSHE'

In [76]:
security_info = get_security_info('000015.XSHE')

In [88]:
security_info == None

True

In [89]:
security_info.start_date

AttributeError: 'NoneType' object has no attribute 'start_date'

In [82]:
price_df = get_price('000002.XSHE', start_date=None, end_date=None, frequency='daily', fields=None, 
                                      skip_paused=False, fq='pre')
price_df

,open,close,high,low,volume,money
2015-01-05,12.42,12.87,13.20,12.28,759952643.0,9.700711e+09
2015-01-06,12.60,12.40,12.94,12.13,387612984.0,4.839616e+09
2015-01-07,12.31,12.29,12.52,12.09,306033250.0,3.772151e+09
2015-01-08,12.36,11.73,12.41,11.62,305725471.0,3.629554e+09
2015-01-09,11.69,11.61,12.28,11.47,381617239.0,4.521978e+09
2015-01-12,11.50,11.33,11.50,11.01,282205524.0,3.180963e+09
2015-01-13,11.27,11.28,11.55,11.20,192814621.0,2.187988e+09
2015-01-14,11.28,11.31,11.48,11.11,190753706.0,2.160268e+09
2015-01-15,11.34,11.89,11.94,11.22,281434697.0,3.265536e+09
2015-01-16,11.89,11.67,11.89,11.56,246660784.0,2.892247e+09


In [3]:
normalized_codes, query_data, error_codes = sd.query_prices("000001,000015")

Could not find price data of this code: 000015


In [4]:
normalized_codes

[]